# WSCC 9bus Ideal Current Source - EMT vs. DP

In [ ]:
import requests
import glob


def download_grid_data(name, url):
    with open(name, "wb") as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)


url = "https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09/WSCC-09"
filename = "WSCC-09"
download_grid_data(filename + "_EQ.xml", url + "_EQ.xml")
download_grid_data(filename + "_TP.xml", url + "_TP.xml")
download_grid_data(filename + "_SV.xml", url + "_SV.xml")

files = glob.glob(filename + "_*.xml")
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import subprocess

PEAK1PH_TO_RMS3PH = np.sqrt(3.0 / 2.0)

name = "DP_WSCC-9bus_IdealVS"
name_emt = "EMT_WSCC-9bus_IdealCS"

timestep = 100e-6
duration = 0.1

root_path = (
    subprocess.Popen(["git", "rev-parse", "--show-toplevel"], stdout=subprocess.PIPE)
    .communicate()[0]
    .rstrip()
    .decode("utf-8")
)
path_exec = root_path + "/build/dpsim/examples/cxx/"

## Run Simulation

In [ ]:
sim = subprocess.Popen(
    [
        path_exec + name,
        "--timestep",
        str(timestep),
        "--duration",
        str(duration),
        files[0],
        files[1],
        files[2],
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
print(sim.communicate()[0].decode())
sim = subprocess.Popen(
    [
        path_exec + name_emt,
        "--timestep",
        str(timestep),
        "--duration",
        str(duration),
        files[0],
        files[1],
        files[2],
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
print(sim.communicate()[0].decode())

## Read DPsim Results

In [ ]:
model_name = "DP_WSCC-9bus_IdealVS"
path = "logs/" + model_name + "/"
dpsim_result_file = path + model_name + ".csv"
ts_dpsim_dp = read_timeseries_csv(dpsim_result_file)

model_name = "EMT_WSCC-9bus_IdealCS"
path = "logs/" + model_name + "/"
dpsim_result_file = path + model_name + ".csv"
ts_dpsim_emt = read_timeseries_csv(dpsim_result_file)

model_name = "EMT_WSCC-9bus_IdealCS_PF"
path = "logs/" + model_name + "/"
dpsim_result_file = path + model_name + ".csv"
ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

## Bus voltages

In [ ]:
plt.figure(figsize=(12, 8))
for phase in ["0"]:  # , '1','2']:
    for name in ["v1_" + phase, "v2_" + phase, "v3_" + phase]:
        plt.plot(
            ts_dpsim_emt[name].time,
            PEAK1PH_TO_RMS3PH * ts_dpsim_emt[name].values,
            label=name + " (EMT)",
        )
for name in ["v1", "v2", "v3"]:
    plt.plot(
        ts_dpsim_dp[name].time, ts_dpsim_dp[name].abs().values, label=name + " (DP)"
    )
for name in ["BUS1.V", "BUS2.V", "BUS3.V"]:
    plt.plot(
        ts_dpsim_pf[name].time,
        ts_dpsim_pf[name].abs().values,
        label=name + " (PF)",
        linestyle="--",
    )
plt.legend()
plt.show()

## Bus angles from DP

In [ ]:
plt.figure(figsize=(12, 8))
for name in ["v1", "v2", "v3"]:
    plt.plot(
        ts_dpsim_dp[name].time, ts_dpsim_dp[name].phase().values, label=name + " (DP)"
    )
for name in ["BUS1.V", "BUS2.V", "BUS3.V"]:
    plt.plot(
        ts_dpsim_pf[name].time,
        ts_dpsim_pf[name].phase().values,
        label=name + " (PF)",
        linestyle="--",
    )
plt.legend()
plt.show()

# Generator currents

In [ ]:
plt.figure(figsize=(12, 8))
for phase in ["0"]:  # ,'1','2']:
    for name in ["GEN1.I_" + phase, "GEN2.I_" + phase, "GEN3.I_" + phase]:
        plt.plot(
            ts_dpsim_emt[name].time,
            PEAK1PH_TO_RMS3PH * ts_dpsim_emt[name].values,
            label=name + "(EMT)",
        )
for name in ["GEN1.I", "GEN2.I", "GEN3.I"]:
    plt.plot(
        ts_dpsim_dp[name].interpolate(timestep).time,
        ts_dpsim_dp[name].interpolate(timestep).frequency_shift(60).values,
        label=name + "(DP)",
        linestyle="--",
    )
plt.legend()
plt.show()

## Assert generator current

In [ ]:
ts_emt_rms3ph = {}
rmse_rel = {}
for name in ["GEN1.I", "GEN2.I", "GEN3.I"]:
    ts_emt_rms3ph[name + "_0"] = ts_dpsim_emt[name + "_0"]
    ts_emt_rms3ph[name + "_0"].values = (
        PEAK1PH_TO_RMS3PH * ts_emt_rms3ph[name + "_0"].values
    )
    rmse_rel[name] = ts_dpsim_dp[name].rmse(
        ts_emt_rms3ph[name + "_0"],
        ts_dpsim_dp[name].interpolate(timestep).frequency_shift(60),
    ) / np.max(ts_dpsim_dp[name].abs().values)
    print("Rel. RMSE for {}: {}".format(name, rmse_rel[name]))
    assert rmse_rel[name] < 1e-2